In [10]:
import httplib2
import os

import requests

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

from pandas import DataFrame, concat, to_datetime, to_numeric
import pandas as pd

import json

# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
CLIENT_SECRET_FILE = 'gsheets_secret.json'
APPLICATION_NAME = 'solarpunktravel backend'

repoSpreadsheetId = '1FHoBYKs-kVpmkmyUi1BA5_qmygJUo0UfiAayc6mqgcc'
prodSpreadsheetId = '1krFtwpYnqe8T7mCaAVJzsqxe_CYDAIbQKwoLMMPZc3k'

In [11]:
from geopy.geocoders import GoogleV3

with open("maps.key", 'r') as f:
    maps_key = f.read()
geolocator = GoogleV3(api_key=maps_key)

with open("hub_umap_urls.json", "r") as infile:
    city_urls = json.load(infile)
    
city_loc = {}
for w in list(city_urls.keys()):
    location = geolocator.geocode(w.split('-')[0])
    if location:
        city_loc[w]= location[1]
        
with open("city_ims.json", "r") as infile:
    city_ims = json.load(infile)

In [12]:
def get_credentials():
    """Gets valid user credentials from storage.
    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.
    Returns:
        Credentials, the obtained credential.
    """
    credential_path = 'sheets.googleapis.com-solarpunktravel-backend.json'

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flags = tools.argparser.parse_args(args=[])
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        credentials = tools.run_flow(flow, store, flags)
        print('Storing credentials to ' + credential_path)
    return credentials

#https://stackoverflow.com/questions/38245714/get-list-of-sheets-and-latest-sheet-in-google-spreadsheet-api-v4-in-python
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

In [6]:
sheet_metadata = service.spreadsheets().get(spreadsheetId=prodSpreadsheetId).execute()
sheets = sheet_metadata.get('sheets', '')
sheet_names = [s.get("properties", {}).get("title", "Sheet1") for s in sheets]

with open(f"hub_umap_urls.json", "r") as infile:
    city_urls = json.load(infile)
city_list =list(city_urls.keys())

missing_cities = list(set(city_list)-set(sheet_names))

for mc in missing_cities:    
    batch_update_spreadsheet_request_body = {
        'requests': [{
          "addSheet": {
            "properties": {
              "title": mc,
            }
          }
        }]
    }

    request = service.spreadsheets().batchUpdate(spreadsheetId=prodSpreadsheetId, body=batch_update_spreadsheet_request_body)
    response = request.execute()


In [14]:

result = service.spreadsheets().values().get(spreadsheetId=repoSpreadsheetId, range="A:J").execute()
raw_data = result.get('values', [])
data_df = DataFrame.from_records([r + [''] * (len(raw_data[0])-len(r)) for r in raw_data[1:]],columns=raw_data[0])

for city, city_df in data_df.groupby('hub'):
    city_pretty = city.replace('_',' ').title()
    post_list = []
    
    ##### do top row
    response = requests.get(f"https://native-land.ca/wp-json/nativeland/v1/api/index.php?maps=territories&position={city_loc[city][0]},{city_loc[city][1]}")
    tribes = [t['properties']['Name'] for t in json.loads(response.content)]
    tribes_string = ', '.join(tribes[:-1]) + ', and ' +tribes[-1]
    city_description = f"Lands of the {tribes_string}"
    
    city_im = city_ims[city][0] if len(city_ims[city])>0 else ''
    
    hub_html = f"""<p>The {city_pretty} Hub occupies the ancestral, traditional and contemporary lands of the {tribes_string}.  We enjoy today what the indigenous people protected and continue to protect, and we remember to be grateful.</p>
    
    <p>Here are some resources to help you travel lightly and well.</p>
    """
    
    # trip groups
    to_add = ''
    for ind, row in city_df.loc[city_df['type']=='trip_group'].iterrows():
        to_add+=f"<li><a href='{row['url']}'>{row['name']}</a></li>"
    if to_add!='':
        hub_html+='<h3>Bike Camping Groups</h3><ul>'+to_add+'</ul>'
    else:
        hub_html+="""<h3>Bike Camping Groups</h3><p><i>This hub doesn't yet have any bike camping groups listed.  If you know of one in this area, help us out by <a onclick="document.getElementById('modform1').style.display='block'" href='#'>submitting a link</a> to their site!</i></p>"""
    
    # volunteer groups
    to_add = ''
    for ind, row in city_df.loc[city_df['type']=='volunteer_group'].iterrows():
        to_add+=f"<li><a href='{row['url']}'>{row['name']}</a></li>"
    if to_add!='':
        hub_html+='<h3>Outdoor Volunteer Orgs</h3><ul>'+to_add+'</ul>'
    else:
        hub_html+="""<h3>Outdoor Volunteer Orgs</h3><p><i>This hub doesn't yet have any outdoor volunteering groups listed.  If you know of one in this area, help us out by <a onclick="document.getElementById('modform1').style.display='block'" href='#'>submitting a link</a> to their site!</i></p>"""
    
    post_list.append([0,'Hub Title Row',city_description,city_im,'NA',hub_html,'NA',''])
    
    #### do journey rows
    for journey, sub_df in city_df.groupby('journey'):
        if journey!='':
            journey_html = ''
            to_add = ''
            desc_l = []
            desc = ''
            for ind, row in sub_df.loc[sub_df['subtype']=='public_camp'].iterrows():
                to_add+=f"<li><a href='{row['url']}'>{row['name']}</a></li>"
                desc_l.append(row['name'])
            if to_add!='':
                journey_html+='<h3>Public Camps</h3><ul>'+to_add+'</ul>'
                if len(desc_l)>1:
                    desc = 'A trip to ' + ', '.join(desc_l[:-1]) + ', and ' +desc_l[-1]
                else:
                    desc = 'A trip to ' +desc_l[0]
            for ind, row in sub_df.loc[sub_df['type']=='route'].iterrows():
                latlon = f"{row['lat']}/{row['lon']}"
                break
            post_list.append([int(journey),f"{city.title()} Journey #{int(journey)+1}",desc,'','NA',journey_html,f"#9/{latlon}",None])
    
    # TODO: make resources journey specific and add them here
    
    #### do log rows
    for journey, sub_df in city_df.groupby('journey'):
        if journey!='':
            for ind, row in sub_df.loc[sub_df['type']=='route'].iterrows():
                post_list.append([None,row['name'],row['description'],row['image'],row['url'],'','NA',int(journey)])
    
#     body = {
#         "majorDimension": "ROWS",
#         "values": [
#             ['index','name','description','image','directions','html','zoom',''],
#             ] + post_list,
#     }
#     resp = service.spreadsheets().values().update(
#         spreadsheetId=prodSpreadsheetId,
#         range=f"{city}!A:H",
#         body=body,
#         valueInputOption='USER_ENTERED'
#     ).execute()
    
    
    ### MD file
    to_write = f"""---
layout: hub
title: {city_pretty}
description: "{city_description}"
group_forum: https://discord.gg/6VbCwAcHwg
spreadsheet_url: https://docs.google.com/spreadsheets/d/{prodSpreadsheetId}/gviz/tq?tqx=out:json&sheet={city}
nav-menu: false
show_tile: false
map: {city_urls[city].split('/')[-1]}
image: {city_im}
---
    
    """ + hub_html
    with open(f"../hubs/{city}.md", "w") as f:
        f.write(to_write)